# Сбор всех игр действующих игроков за сезон 2024-25

Цель: получить все игровые записи (game logs) для игроков, которые сыграли хотя бы в одной игре в сезоне 2024-25.

Краткий контракт:
- Ввод: сезон в формате `'2024-25'`.
- Выход: DataFrame со всеми матчами игроков, которые имели хотя бы один матч в сезоне, и CSV `games_2024-25_active_players.csv`.
- Ошибки сети/API: реализована простая повторная попытка и пауза между запросами (rate-limit).

In [2]:
# Установка зависимостей (в ноутбуке — выполнится в ячейке)
import sys
try:
    from nba_api.stats.static import players, teams as teams_static
    from nba_api.stats.endpoints import playergamelog, commonallplayers, commonteamroster
    import pandas as pd
    # Используем auto-версию tqdm — она автоматически выберет корректный рендерер для Jupyter/VS Code/терминала
    from tqdm.auto import tqdm
except Exception as e:
    # Попробуем установить зависимости и повторно импортировать
    import subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--upgrade', 'nba_api', 'pandas', 'tqdm', 'ipywidgets'])
    from nba_api.stats.static import players, teams as teams_static
    from nba_api.stats.endpoints import playergamelog, commonallplayers, commonteamroster
    import pandas as pd
    from tqdm.auto import tqdm
import time
import os

# Helper: получить список игроков через составы команд для указанного сезона
def get_players_from_team_rosters(season='2024-25'):
    """Собирает игроков, идущих в составах команд сезона (CommonTeamRoster для каждой команды).
    Это даёт корректный список активных игроков в сезоне (обычно ~400-500), в отличие от CommonAllPlayers,
    который иногда возвращает всех исторических игроков.
    Возвращает список словарей {'id': PLAYER_ID, 'full_name': PLAYER}.
    """
    try:
        teams_list = teams_static.get_teams()
        players_map = {}
        for t in teams_list:
            team_id = t.get('id')
            try:
                roster_df = commonteamroster.CommonTeamRoster(team_id=team_id, season=season).get_data_frames()[0]
            except Exception:
                # Пропускаем команду если не удалось получить состав
                continue
            # CommonTeamRoster возвращает столбцы PLAYER_ID и PLAYER (имя)
            for _, row in roster_df.iterrows():
                try:
                    pid = int(row.get('PLAYER_ID'))
                    name = row.get('PLAYER') or row.get('DISPLAY_FIRST_LAST')
                    players_map[pid] = name
                except Exception:
                    continue
        return [{'id': pid, 'full_name': name} for pid, name in players_map.items()]
    except Exception:
        return []

# Сохранённая реализация как fallback: использует CommonAllPlayers (иногда возвращает много записей)
def get_all_players(season='2024-25'):
    """Попытка получить список игроков через составы команд; при неудаче — fallback на CommonAllPlayers.
    """
    players_list = get_players_from_team_rosters(season=season)
    if players_list:
        return players_list
    # fallback — попробуем CommonAllPlayers
    try:
        df = commonallplayers.CommonAllPlayers(season=season, league_id='00').get_data_frames()[0]
        players_list = []
        for _, r in df.iterrows():
            players_list.append({'id': int(r.get('PERSON_ID')), 'full_name': r.get('DISPLAY_FIRST_LAST')})
        return players_list
    except Exception:
        return []

# Helper: получить gamelog для игрока в указанном сезоне
def fetch_player_gamelog(player_id, season='2024-25', season_type='Regular Season', max_retries=3):
    """Вернёт DataFrame с игровыми записями игрока за сезон или пустой DataFrame, если записей нет.
    Обрабатывает простые ошибки сети с повторной попыткой.
    """
    for attempt in range(max_retries):
        try:
            gl = playergamelog.PlayerGameLog(player_id=player_id, season=season, season_type_all_star=season_type)
            df = gl.get_data_frames()[0]
            return df
        except Exception as e:
            # Небольшая экспоненциальная задержка перед повтором
            backoff = 1 + attempt * 2
            time.sleep(backoff)
    # Если не удалось получить данные — вернуть пустой DataFrame с ожидаемыми колонками (если нужно)
    return pd.DataFrame()

# Основная функция: получить все игры для игроков, которые играли в сезоне
def get_active_players_games(season='2024-25', out_csv='games_2024-25_active_players.csv', sleep_between_calls=0.6, limit_players=None):
    # Явно запрашиваем игроков для указанного сезона, чтобы не брать всех исторических игроков
    print(f"Fetching active players for season {season}...")
    all_players = get_all_players(season=season)
    print(f"Total active players found: {len(all_players)}")
    # Отладочная информация: сколько игроков вернул сбор по составам команд для этого сезона
    print(f"Players returned by team rosters / CommonAllPlayers fallback for {season}: {len(all_players)}")
    # Опционально ограничим количество игроков (полезно для теста)
    if limit_players is not None:
        all_players = all_players[:limit_players]

    collected = []
    # Пробегаем по всем игрокам и собираем их gamelogs, если они не пустые
    for p in tqdm(all_players, desc=f'Players (checked)'):
        pid = p.get('id') or p.get('player_id')
        name = p.get('full_name')
        df = fetch_player_gamelog(pid, season=season)
        if df is not None and not df.empty:
            # Добавим имя/ид игрока в строку, чтобы потом было удобнее фильтровать/агрегировать
            df = df.copy()
            df['PLAYER_NAME'] = name
            df['PLAYER_ID'] = pid
            collected.append(df)
        # Пауза чтобы уменьшить риск бана/rate-limit
        time.sleep(sleep_between_calls)

    if len(collected) == 0:
        result = pd.DataFrame()
    else:
        result = pd.concat(collected, ignore_index=True, sort=False)

    # Сохраняем в CSV, если требуется
    if out_csv and not result.empty:
        result.to_csv(out_csv, index=False, encoding='utf-8')
    return result

In [3]:
# Пример вызова: собрать все игры за сезон 2024-25 (внимание: выполнение займёт время и зависит от API rate limits).
season = '2024-25'
# Для теста можно поставить limit_players=50 или подобное, затем убрать ограничение для полного сбора.
df_games = get_active_players_games(season=season, out_csv='games_2024-25_active_players.csv', sleep_between_calls=0.6, limit_players=None)

# Покажем результат
print('Total games rows fetched:', len(df_games))
# Отобразить первые строки (в Jupyter это покажет таблицу)
df_games.head()

Fetching active players for season 2024-25...
Total active players found: 499
Players returned by team rosters / CommonAllPlayers fallback for 2024-25: 499


Players (checked):   0%|          | 0/499 [00:00<?, ?it/s]

Total games rows fetched: 24081


,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,PLAYER_NAME,PLAYER_ID
0,22024,1631210,0022401186,"Apr 13, 2025",ATL vs. ORL,W,27,6,12,0.5,...,2,0,0,1,0,17,-7,1,Jacob Toppin,1631210
1,22024,1631210,0022400692,"Feb 01, 2025",NYK vs. LAL,L,2,0,1,0.0,...,0,0,0,0,0,0,0,1,Jacob Toppin,1631210
2,22024,1631210,0022400653,"Jan 27, 2025",NYK vs. MEM,W,4,0,0,0.0,...,0,0,0,0,1,0,3,1,Jacob Toppin,1631210
3,22024,1631210,0022400641,"Jan 25, 2025",NYK vs. SAC,W,3,0,0,0.0,...,0,1,0,0,0,0,2,1,Jacob Toppin,1631210
4,22024,1631210,0022400539,"Jan 12, 2025",NYK vs. MIL,W,5,1,2,0.5,...,0,1,0,1,1,2,4,1,Jacob Toppin,1631210


In [5]:
df_games.to_csv('games_2024-25_active_players.csv', index=False, encoding='utf-8')

In [ ]:
df_games

517